In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install google-api-python-client


In [3]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import os
import io

# Authenticate and create the Google Drive service
auth.authenticate_user()

from google.auth import default

creds, _ = default()

service = build('drive', 'v3', credentials=creds)


In [4]:
import os
import io

# Function to list files in a Google Drive folder
def list_files_in_folder(service, folder_id):
    results = []
    page_token = None

    while True:
        response = service.files().list(
            q=f"'{folder_id}' in parents",
            spaces='drive',
            fields='nextPageToken, files(id, name)',
            pageToken=page_token
        ).execute()
        items = response.get('files', [])
        results.extend(items)
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return results

# Function to download files from Google Drive
def download_files(service, items, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for item in items:
        request = service.files().get_media(fileId=item['id'])
        fh = io.FileIO(os.path.join(destination_folder, item['name']), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        print(f"Downloaded {item['name']}")


folder_id = '1IVspCwRF6V2OAj-ynZ2chDjjOWhiwLiJ'  #folder ID
destination_folder = '/content/drive/MyDrive/untitled7_data'  #destination folder

# List and download files
items = list_files_in_folder(service, folder_id)
download_files(service, items, destination_folder)


HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1f9YdNPt_qciqqaJjddykZgAwJ3ef5HfS?alt=media returned "Only files with binary content can be downloaded. Use Export with Docs Editors files.". Details: "[{'message': 'Only files with binary content can be downloaded. Use Export with Docs Editors files.', 'domain': 'global', 'reason': 'fileNotDownloadable', 'location': 'alt', 'locationType': 'parameter'}]">